In [1]:
!pip install google-cloud-pubsub
!pip install pandas
# ! gcloud auth login
# ! gcloud auth application-default login

### Python Client for Google Cloud Pub / Sub
https://googleapis.dev/python/pubsub/latest/index.html

In [1]:
from google.cloud import pubsub
import pandas as pd
from google.cloud import pubsub_v1

### Set options

In [26]:
#!gcloud pubsub topics create my-topic3
#!gcloud pubsub subscriptions create my-sub3 --topic my-topic2

Created subscription [projects/jutaku-sandbox/subscriptions/my-sub3].


In [38]:
topic_name = 'my-topic'
subscription_name = 'my-subscription'
input_path = './sensor_obs2008.csv.gz'
project_id = ''

###  Read Data

In [36]:
!gsutil cp gs://cloud-training-demos/sandiego/sensor_obs2008.csv.gz .
data  =  pd.read_csv(input_path)
data.head()

Copying gs://cloud-training-demos/sandiego/sensor_obs2008.csv.gz...
/ [1 files][ 34.6 MiB/ 34.6 MiB]                                                
Operation completed over 1 objects/34.6 MiB.                                     


,TIMESTAMP,LATITUDE,LONGITUDE,FREEWAY_ID,FREEWAY_DIR,LANE,SPEED
0,2008-11-01 00:00:00,32.749679,-117.155519,163,S,1,71.2
1,2008-11-01 00:00:00,32.749679,-117.155519,163,S,2,65.1
2,2008-11-01 00:00:00,32.780922,-117.089026,8,W,1,76.5
3,2008-11-01 00:00:00,32.780922,-117.089026,8,W,2,74.0
4,2008-11-01 00:00:00,32.780922,-117.089026,8,W,3,72.0


### Publish message

In [19]:
publisher = pubsub_v1.PublisherClient()
#topic_path = publisher.topic_path(project_id, topic_name)
topic_name = 'projects/{project_id}/topics/{topic}'.format(
    project_id=project_id,
    topic=topic_name,  # Set this to something appropriate.
)
publisher.create_topic(topic_name)

/home/keisuke_taniguchi/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


name: "projects/jutaku-sandbox/topics/my-topic2"

In [34]:
for i , v in data.head(10).iterrows():
    message = str(list(v.values)).encode('utf-8')
    future = publisher.publish(topic_name, data=message)
    print(message)

b"['2008-11-01 00:00:00', 32.749679, -117.155519, 163, 'S', 1, 71.2]"
b"['2008-11-01 00:00:00', 32.749679, -117.155519, 163, 'S', 2, 65.1]"
b"['2008-11-01 00:00:00', 32.780922, -117.089026, 8, 'W', 1, 76.5]"
b"['2008-11-01 00:00:00', 32.780922, -117.089026, 8, 'W', 2, 74.0]"
b"['2008-11-01 00:00:00', 32.780922, -117.089026, 8, 'W', 3, 72.0]"
b"['2008-11-01 00:00:00', 32.780922, -117.089026, 8, 'W', 4, 66.8]"
b"['2008-11-01 00:00:00', 32.780922, -117.089026, 8, 'W', 5, 64.5]"
b"['2008-11-01 00:00:00', 32.779415, -117.06815, 8, 'W', 1, 76.5]"
b"['2008-11-01 00:00:00', 32.779415, -117.06815, 8, 'W', 2, 72.3]"
b"['2008-11-01 00:00:00', 32.779415, -117.06815, 8, 'W', 3, 72.0]"
b"['2008-11-01 00:00:00', 32.749679, -117.155519, 163, 'S', 1, 71.2]"b"['2008-11-01 00:00:00', 32.749679, -117.155519, 163, 'S', 2, 65.1]"
b"['2008-11-01 00:00:00', 32.780922, -117.089026, 8, 'W', 1, 76.5]"
b"['2008-11-01 00:00:00', 32.780922, -117.089026, 8, 'W', 2, 74.0]"
b"['2008-11-01 00:00:00', 32.780922, -117.08

### Subscribe message

In [28]:
subscriber = pubsub_v1.SubscriberClient()
subscription_name = 'projects/{project_id}/subscriptions/{sub}'.format(
    project_id=project_id,
    sub=subscription_name ,  # Set this to something appropriate.
)
subscriber.create_subscription(
    name=subscription_name, topic=topic_name)

/home/keisuke_taniguchi/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


name: "projects/jutaku-sandbox/subscriptions/my-subscription4"
topic: "projects/jutaku-sandbox/topics/my-topic2"
push_config {
}
ack_deadline_seconds: 10
message_retention_duration {
  seconds: 604800
}
expiration_policy {
  ttl {
    seconds: 2678400
  }
}

In [35]:
def callback(message):
    print(message.data)
    message.ack()
future = subscriber.subscribe(subscription_name, callback)

b"['2008-11-01 00:00:00', 32.749679, -117.155519, 163, 'S', 1, 71.2]"
b"['2008-11-01 00:00:00', 32.749679, -117.155519, 163, 'S', 2, 65.1]"
b"['2008-11-01 00:00:00', 32.780922, -117.089026, 8, 'W', 1, 76.5]"
b"['2008-11-01 00:00:00', 32.780922, -117.089026, 8, 'W', 2, 74.0]"
b"['2008-11-01 00:00:00', 32.780922, -117.089026, 8, 'W', 3, 72.0]"
b"['2008-11-01 00:00:00', 32.780922, -117.089026, 8, 'W', 4, 66.8]"
b"['2008-11-01 00:00:00', 32.780922, -117.089026, 8, 'W', 5, 64.5]"
b"['2008-11-01 00:00:00', 32.779415, -117.06815, 8, 'W', 1, 76.5]"
b"['2008-11-01 00:00:00', 32.779415, -117.06815, 8, 'W', 2, 72.3]"
b"['2008-11-01 00:00:00', 32.779415, -117.06815, 8, 'W', 3, 72.0]"
b"['2008-11-01 00:00:00', 32.749679, -117.155519, 163, 'S', 1, 71.2]"b"['2008-11-01 00:00:00', 32.749679, -117.155519, 163, 'S', 2, 65.1]"b"['2008-11-01 00:00:00', 32.780922, -117.089026, 8, 'W', 1, 76.5]"
b"['2008-11-01 00:00:00', 32.780922, -117.089026, 8, 'W', 2, 74.0]"
b"['2008-11-01 00:00:00', 32.780922, -117.089